In [ ]:
import pandas as pd

df = pd.read_csv("C:\Users\manid\OneDrive\Desktop\AI Enabled Visa Status Prediction and Processing Time Estimator\dataset\us_perm_visas.csv", low_memory=False)

df.shape
df.columns


Index(['add_these_pw_job_title_9089', 'agent_city', 'agent_firm_name',
       'agent_state', 'application_type', 'case_no', 'case_number',
       'case_received_date', 'case_status', 'class_of_admission',
       ...
       'ri_pvt_employment_firm_to', 'ri_us_workers_considered',
       'schd_a_sheepherder', 'us_economic_sector', 'wage_offer_from_9089',
       'wage_offer_to_9089', 'wage_offer_unit_of_pay_9089',
       'wage_offered_from_9089', 'wage_offered_to_9089',
       'wage_offered_unit_of_pay_9089'],
      dtype='object', length=154)

In [ ]:
df.head()

,add_these_pw_job_title_9089,agent_city,agent_firm_name,agent_state,application_type,case_no,case_number,case_received_date,case_status,class_of_admission,...,ri_pvt_employment_firm_to,ri_us_workers_considered,schd_a_sheepherder,us_economic_sector,wage_offer_from_9089,wage_offer_to_9089,wage_offer_unit_of_pay_9089,wage_offered_from_9089,wage_offered_to_9089,wage_offered_unit_of_pay_9089
0,NaN,NaN,NaN,NaN,PERM,A-07323-97014,NaN,NaN,Certified,J-1,...,NaN,NaN,NaN,IT,75629.0,NaN,yr,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,PERM,A-07332-99439,NaN,NaN,Denied,B-2,...,NaN,NaN,NaN,Other Economic Sector,37024.0,NaN,yr,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,PERM,A-07333-99643,NaN,NaN,Certified,H-1B,...,NaN,NaN,NaN,Aerospace,47923.0,NaN,yr,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,PERM,A-07339-01930,NaN,NaN,Certified,B-2,...,NaN,NaN,NaN,Other Economic Sector,10.97,NaN,hr,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,PERM,A-07345-03565,NaN,NaN,Certified,L-1,...,NaN,NaN,NaN,Advanced Mfg,100000.0,NaN,yr,NaN,NaN,NaN


In [ ]:
df[['case_received_date','decision_date']].isnull().sum()

,0
case_received_date,135269
decision_date,0


In [ ]:
df.isnull().sum()

,0
add_these_pw_job_title_9089,137496
agent_city,142453
agent_firm_name,143390
agent_state,142966
application_type,44421
...,...
wage_offer_to_9089,163121
wage_offer_unit_of_pay_9089,115164
wage_offered_from_9089,65043
wage_offered_to_9089,148675


In [ ]:
df['case_received_date'] = pd.to_datetime(df['case_received_date'], errors='coerce')
df['decision_date'] = pd.to_datetime(df['decision_date'], errors='coerce')

In [ ]:
df[['case_received_date','decision_date']].isnull().sum()

,0
case_received_date,135269
decision_date,0


In [ ]:
df_clean = df[
    (df['case_received_date'].notna()) &
    (df['decision_date'].notna())
]

df_clean = df_clean[
    df_clean['decision_date'] >= df_clean['case_received_date']
]
df_clean.shape


(44421, 154)

In [ ]:
df_clean['processing_days'] = (
    df_clean['decision_date'] - df_clean['case_received_date']
).dt.days

In [ ]:
print(df_clean['processing_days'].describe())

count    44421.000000
mean       346.185836
std        260.077149
min          0.000000
25%        149.000000
50%        206.000000
75%        526.000000
max       3355.000000
Name: processing_days, dtype: float64


In [ ]:
lower = df_clean['processing_days'].quantile(0.01)
upper = df_clean['processing_days'].quantile(0.99)

df_clean = df_clean[
    (df_clean['processing_days'] >= lower) &
    (df_clean['processing_days'] <= upper)
]

print("After Outlier Removal:")
df_clean.shape

After Outlier Removal:


(43546, 155)

In [ ]:
important_cols = [
    'country_of_citizenship',
    'class_of_admission',
    'employer_state',
    'us_economic_sector',
    'case_status',
    'foreign_worker_info_education',
    'foreign_worker_info_major',
    'case_received_date',
    'decision_date',
    'processing_days'
]

df_clean = df_clean[important_cols]

In [ ]:
for col in df_clean.select_dtypes(include='object').columns:
    df_clean[col] = df_clean[col].fillna("UNKNOWN")

In [ ]:
for col in df_clean.select_dtypes(include='object').columns:
    df_clean[col] = df_clean[col].str.strip().str.upper()

In [ ]:
df_clean['application_year'] = df_clean['case_received_date'].dt.year
df_clean['application_month'] = df_clean['case_received_date'].dt.month
df_clean['application_quarter'] = df_clean['case_received_date'].dt.quarter
df_clean['application_dayofweek'] = df_clean['case_received_date'].dt.dayofweek

In [ ]:
final_status = ['CERTIFIED', 'DENIED']
df_clean = df_clean[df_clean['case_status'].isin(final_status)]

In [ ]:
df_clean = df_clean.drop_duplicates()
print("Final Shape:")
df_clean.shape

Final Shape:


(14622, 14)

In [ ]:
df_clean.isnull().sum()

,0
country_of_citizenship,0
class_of_admission,0
employer_state,0
us_economic_sector,0
case_status,0
foreign_worker_info_education,0
foreign_worker_info_major,0
case_received_date,0
decision_date,0
processing_days,0


In [ ]:
df_clean.to_csv("cleaned_us_visa_dataset.csv", index=False)